# Librerias

In [1]:
import pandas as pd
import numpy as np

In [2]:
from datetime import datetime, timedelta
pd.set_option('display.max_rows', None)
import matplotlib.pyplot as plt
import seaborn as sns
pd.set_option('display.max_columns',None)
import warnings
warnings.filterwarnings("ignore")

In [3]:
from sklearn.preprocessing import StandardScaler , OrdinalEncoder, LabelEncoder

# Funciones

# Importar Bases de Datos

In [4]:
calendario = pd.read_csv("data/daily_calendar_with_events.csv")
precios = pd.read_csv("data/item_prices.csv")
ventas = pd.read_csv("data/item_sales.csv")

# Preproccesing

## Tabla Ventas Modificada

In [5]:
ventas.groupby(["region","store_code","store"]).agg({"d_1":"count"})

d_1
region       store_code store                  
Boston       BOS_1      South_End          3049
             BOS_2      Roxbury            3049
             BOS_3      Back_Bay           3049
New York     NYC_1      Greenwich_Village  3049
             NYC_2      Harlem             3049
             NYC_3      Tribeca            3049
             NYC_4      Brooklyn           3049
Philadelphia PHI_1      Midtown_Village    3049
             PHI_2      Yorktown           3049
             PHI_3      Queen_Village      3049

In [6]:
ultimodia=int(str(ventas.columns.to_list()[-1]).split("_")[1]) # buscamos el ultimo dia (ultima columna del DF)
lista=[ultimodia+x for x in range(1,29)] # creamos una lista con los siguientes 28 dias

for i in lista:
    colname="d_"+str(i)
    ventas[colname]=0 #creamos una columna por cada dia adicional con ventas 0.

idlist=[]
LE=LabelEncoder()
iditem =LE.fit(ventas["item"])
idlist.append({l: i for (i, l) in enumerate(iditem.classes_)})
idcat =LE.fit(ventas["category"])
idlist.append( {l: i for (i, l) in enumerate(idcat.classes_)})
iddep = LE.fit(ventas["department"])
idlist.append({l: i for (i, l) in enumerate(iddep.classes_)})
idsto =LE.fit(ventas["store"])
idlist.append({l: i for (i, l) in enumerate(idsto.classes_)})
idstocode =LE.fit(ventas["store_code"])
idlist.append({l: i for (i, l) in enumerate(idstocode.classes_)})
idreg =LE.fit(ventas["region"])
idlist.append({l: i for (i, l) in enumerate(idreg.classes_)})

for i in zip(ventas.columns[1:7].tolist(),idlist):
    
    ventas[i[0]]= ventas[i[0]].replace(i[1]).astype(int)


ventasmelt=ventas.melt(id_vars=ventas.columns[:7],value_vars=ventas.columns[7:],var_name="d",value_name="Cantidad") #Transformacion de BD, dias por Fila

#verificamos nulos
if len(ventasmelt.isnull().sum()[ventasmelt.isnull().sum() > 0]) ==0:
    print("Sin nulos")
else:
    print("Columnas con nulos: ", ventasmelt.isnull().sum()[ventasmelt.isnull().sum() > 0])

#verificamos filas duplicadas, las eliminamos y reseteamos el indice
print("Numero de filas duplicadas: ",len(ventasmelt[ventasmelt.duplicated(keep=False)]))
ventasmelt.drop_duplicates(inplace=True)
ventasmelt.reset_index(drop=True,inplace=True)

print("Suma de ventas totales: ",ventasmelt["Cantidad"].sum())
print("Observaciones con ventas 0: ",len(ventasmelt[ventasmelt["Cantidad"]==0].index))

for i in ventasmelt:
    if i !="Cantidad" and i !="d" :
        print(f"Numero de {i} unicos",ventasmelt[i].nunique())

print("Dimensiones DF tratado: ",ventasmelt.shape)


Sin nulos
Numero de filas duplicadas:  0
Suma de ventas totales:  65695409
Observaciones con ventas 0:  40630814
Numero de id unicos 30490
Numero de item unicos 3049
Numero de category unicos 3
Numero de department unicos 7
Numero de store unicos 10
Numero de store_code unicos 10
Numero de region unicos 3
Dimensiones DF tratado:  (59181090, 9)


In [7]:
idlist

[{'ACCESORIES_1_001': 0,
  'ACCESORIES_1_002': 1,
  'ACCESORIES_1_003': 2,
  'ACCESORIES_1_004': 3,
  'ACCESORIES_1_005': 4,
  'ACCESORIES_1_006': 5,
  'ACCESORIES_1_007': 6,
  'ACCESORIES_1_008': 7,
  'ACCESORIES_1_009': 8,
  'ACCESORIES_1_010': 9,
  'ACCESORIES_1_011': 10,
  'ACCESORIES_1_012': 11,
  'ACCESORIES_1_013': 12,
  'ACCESORIES_1_014': 13,
  'ACCESORIES_1_015': 14,
  'ACCESORIES_1_016': 15,
  'ACCESORIES_1_017': 16,
  'ACCESORIES_1_018': 17,
  'ACCESORIES_1_019': 18,
  'ACCESORIES_1_020': 19,
  'ACCESORIES_1_021': 20,
  'ACCESORIES_1_022': 21,
  'ACCESORIES_1_023': 22,
  'ACCESORIES_1_024': 23,
  'ACCESORIES_1_025': 24,
  'ACCESORIES_1_026': 25,
  'ACCESORIES_1_027': 26,
  'ACCESORIES_1_028': 27,
  'ACCESORIES_1_029': 28,
  'ACCESORIES_1_030': 29,
  'ACCESORIES_1_031': 30,
  'ACCESORIES_1_032': 31,
  'ACCESORIES_1_033': 32,
  'ACCESORIES_1_034': 33,
  'ACCESORIES_1_035': 34,
  'ACCESORIES_1_036': 35,
  'ACCESORIES_1_037': 36,
  'ACCESORIES_1_038': 37,
  'ACCESORIES_1_039': 

In [8]:
ventas=ventasmelt.copy()

In [9]:
ventas.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 59181090 entries, 0 to 59181089
Data columns (total 9 columns):
 #   Column      Dtype 
---  ------      ----- 
 0   id          object
 1   item        int32 
 2   category    int32 
 3   department  int32 
 4   store       int32 
 5   store_code  int32 
 6   region      int32 
 7   d           object
 8   Cantidad    int64 
dtypes: int32(6), int64(1), object(2)
memory usage: 2.6+ GB


In [10]:
ventas.head()

,id,item,category,department,store,store_code,region,d,Cantidad
0,ACCESORIES_1_001_NYC_1,0,0,0,2,3,1,d_1,0
1,ACCESORIES_1_002_NYC_1,1,0,0,2,3,1,d_1,0
2,ACCESORIES_1_003_NYC_1,2,0,0,2,3,1,d_1,0
3,ACCESORIES_1_004_NYC_1,3,0,0,2,3,1,d_1,0
4,ACCESORIES_1_005_NYC_1,4,0,0,2,3,1,d_1,0


## Tabla Calendario Modificada

In [11]:
calendario.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1913 entries, 0 to 1912
Data columns (total 5 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   date         1913 non-null   object
 1   weekday      1913 non-null   object
 2   weekday_int  1913 non-null   int64 
 3   d            1913 non-null   object
 4   event        26 non-null     object
dtypes: int64(1), object(4)
memory usage: 74.9+ KB


In [12]:
calendario["date"]=pd.to_datetime(calendario["date"]) # formato fecha en columna date

In [13]:
calendario.sort_values(["date"], ascending = True, inplace = True)

In [14]:
calendario.tail(30)

,date,weekday,weekday_int,d,event
1883,2016-03-26,Saturday,1,d_1884,NaN
1884,2016-03-27,Sunday,2,d_1885,Easter
1885,2016-03-28,Monday,3,d_1886,NaN
1886,2016-03-29,Tuesday,4,d_1887,NaN
1887,2016-03-30,Wednesday,5,d_1888,NaN
1888,2016-03-31,Thursday,6,d_1889,NaN
1889,2016-04-01,Friday,7,d_1890,NaN
1890,2016-04-02,Saturday,1,d_1891,NaN
1891,2016-04-03,Sunday,2,d_1892,NaN
1892,2016-04-04,Monday,3,d_1893,NaN


In [15]:
print("Dimensiones DF original: ",calendario.shape)

#agregamos los ultimos 28 dias a proyectar
date_offset = pd.DateOffset(days=1)
ultimafila=calendario.iloc[-1:]
for dia in lista:
    new_row = ultimafila.copy()
    new_row['d'] = "d_"+str(dia)
    new_row['date'] = ultimafila["date"]+date_offset
    new_row['weekday']=new_row['date'].dt.day_name()
    new_row['weekday_int']=np.nan
    new_row['event']=np.nan
    ultimafila=new_row.copy()
    calendario = pd.concat([calendario, new_row], ignore_index=True)


#verificamos nulos
if len(calendario.isnull().sum()[calendario.isnull().sum() > 0]) ==0:
    print("Sin nulos")
else:
    print("Columnas con nulos: ", calendario.isnull().sum()[calendario.isnull().sum() > 0])

#verificamos filas duplicadas, las eliminamos y reseteamos el indice
print("Numero de filas duplicadas: ",len(calendario[calendario.duplicated(keep=False)]))
calendario.drop_duplicates(inplace=True)
calendario.reset_index(drop=True,inplace=True)

calendario["date"]=pd.to_datetime(calendario["date"]) # formato fecha en columna date

# calendario['week'] = calendario['date'].dt.isocalendar().week
# calendario['day'] = calendario['date'].dt.day
# calendario['month'] = calendario['date'].dt.month
calendario['year'] = calendario['date'].dt.year

# concatena semana y año en un string, agregando el 0 a semanas del 1 al 9
calendario['week'] = calendario['date'].dt.isocalendar().week.astype(str)
for i in range(len(calendario)):
    if int(calendario.loc[i,'week']) < 10:
        calendario.loc[i,'week'] = '0'+calendario.loc[i,'week'] #.apply(lambda x: '0{}'.format(x) if x < 10 else str(x))
calendario['year'] = calendario['date'].dt.isocalendar().year
calendario['yearweek']=calendario['year'].astype(str)+calendario['week']

#rellenamos nulos con 0 y pasamos dias con eventos a 1.
calendario['event2'] = calendario['event']
#calendario['event2'] = calendario['event'].apply(lambda x: x if x == 1 else None)  # Establecer a None excepto para los días de evento
calendario['event2'] = calendario['event2'].fillna(method='bfill', limit=6)  # Rellenar hacia atrás hasta 7 días

calendario.loc[calendario['date'].dt.month == 12, 'event2'] = 1 #Rellenar diciembre con 1
calendario["event2"]=np.where(calendario["event2"].isnull(),0,1)
calendario["event"]=np.where(calendario["event"].isnull(),0,1)

#verificamos nulos
if len(calendario.isnull().sum()[calendario.isnull().sum() > 0]) ==0:
    print("Sin nulos")
else:
    print("Columnas con nulos: ", calendario.isnull().sum()[calendario.isnull().sum() > 0])

print("Numero de filas duplicadas: ",len(calendario[calendario.duplicated(keep=False)]))

print("Dimensiones DF tratado: ",calendario.shape)

Dimensiones DF original:  (1913, 5)
Columnas con nulos:  weekday_int      28
event          1915
dtype: int64
Numero de filas duplicadas:  0
Columnas con nulos:  weekday_int    28
dtype: int64
Numero de filas duplicadas:  0
Dimensiones DF tratado:  (1941, 9)


In [16]:
calendario.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1941 entries, 0 to 1940
Data columns (total 9 columns):
 #   Column       Non-Null Count  Dtype         
---  ------       --------------  -----         
 0   date         1941 non-null   datetime64[ns]
 1   weekday      1941 non-null   object        
 2   weekday_int  1913 non-null   float64       
 3   d            1941 non-null   object        
 4   event        1941 non-null   int32         
 5   year         1941 non-null   UInt32        
 6   week         1941 non-null   object        
 7   yearweek     1941 non-null   object        
 8   event2       1941 non-null   int32         
dtypes: UInt32(1), datetime64[ns](1), float64(1), int32(2), object(4)
memory usage: 115.8+ KB


In [21]:
calendario

,date,weekday,weekday_int,d,event,year,week,yearweek,event2
0,2011-01-29,Saturday,1.0,d_1,0,2011,04,201104,0
1,2011-01-30,Sunday,2.0,d_2,0,2011,04,201104,0
2,2011-01-31,Monday,3.0,d_3,0,2011,05,201105,1
3,2011-02-01,Tuesday,4.0,d_4,0,2011,05,201105,1
4,2011-02-02,Wednesday,5.0,d_5,0,2011,05,201105,1
5,2011-02-03,Thursday,6.0,d_6,0,2011,05,201105,1
6,2011-02-04,Friday,7.0,d_7,0,2011,05,201105,1
7,2011-02-05,Saturday,1.0,d_8,0,2011,05,201105,1
8,2011-02-06,Sunday,2.0,d_9,1,2011,05,201105,1
9,2011-02-07,Monday,3.0,d_10,0,2011,06,201106,0


In [18]:
calendario.tail(60)

,date,weekday,weekday_int,d,event,year,week,yearweek,event2
1881,2016-03-24,Thursday,6.0,d_1882,0,2016,12,201612,1
1882,2016-03-25,Friday,7.0,d_1883,0,2016,12,201612,1
1883,2016-03-26,Saturday,1.0,d_1884,0,2016,12,201612,1
1884,2016-03-27,Sunday,2.0,d_1885,1,2016,12,201612,1
1885,2016-03-28,Monday,3.0,d_1886,0,2016,13,201613,0
1886,2016-03-29,Tuesday,4.0,d_1887,0,2016,13,201613,0
1887,2016-03-30,Wednesday,5.0,d_1888,0,2016,13,201613,0
1888,2016-03-31,Thursday,6.0,d_1889,0,2016,13,201613,0
1889,2016-04-01,Friday,7.0,d_1890,0,2016,13,201613,0
1890,2016-04-02,Saturday,1.0,d_1891,0,2016,13,201613,0


In [19]:
calendario[(calendario["week"]=="17")|(calendario["week"]=="18")|(calendario["week"]=="19")|(calendario["week"]=="20")]["event"]

86      0
87      0
88      0
89      0
90      0
91      0
92      0
93      0
94      0
95      0
96      0
97      0
98      0
99      0
100     0
101     0
102     0
103     0
104     0
105     0
106     0
107     0
108     0
109     0
110     0
111     0
112     0
113     0
450     0
451     0
452     0
453     0
454     0
455     0
456     0
457     0
458     0
459     0
460     0
461     0
462     0
463     0
464     0
465     0
466     0
467     0
468     0
469     0
470     0
471     0
472     0
473     0
474     0
475     0
476     0
477     0
814     0
815     0
816     0
817     0
818     0
819     0
820     0
821     0
822     0
823     0
824     0
825     0
826     0
827     0
828     0
829     0
830     0
831     0
832     0
833     0
834     0
835     0
836     0
837     0
838     0
839     0
840     0
841     0
1178    0
1179    0
1180    0
1181    0
1182    0
1183    0
1184    0
1185    0
1186    0
1187    0
1188    0
1189    0
1190    0
1191    0
1192    0
1193    0


## Tabla Precios Modificada

In [ ]:
precios.head()

In [ ]:
precios["yearweek"]=precios["yearweek"].astype(str)
precios["yearweek"]=precios["yearweek"].str.slice(stop=6)

In [ ]:
print("Dimensiones DF original: ",precios.shape)

precios["id"]=precios["item"]+"_"+precios["store_code"] # crea id coditem+codstore



#verificamos nulos
if len(precios.isnull().sum()[precios.isnull().sum() > 0]) ==0:
    print("Sin nulos")
else:
    print("Columnas con nulos: ", precios.isnull().sum()[precios.isnull().sum() > 0])

#verificamos filas duplicadas, las eliminamos y reseteamos el indice
print("Numero de filas duplicadas: ",len(precios[precios.duplicated(keep=False)]))
precios.drop_duplicates(inplace=True)
precios.reset_index(drop=True,inplace=True)

#verificamos nulos
if len(precios.isnull().sum()[precios.isnull().sum() > 0]) ==0:
    print("Sin nulos")
else:
    print("Columnas con nulos: ", precios.isnull().sum()[precios.isnull().sum() > 0])

precios["yearweek"]=precios["yearweek"].astype(str)
precios["yearweek"]=precios["yearweek"].str.slice(stop=6) # quita .0 del string
# precios["year"]=precios["yearweek"].str.slice(stop=4) # crea columna con año
# precios["semana"]=precios["yearweek"].str.slice(start=4,stop=6) # crea columna con semana





print("Dimensiones DF tratado: ",precios.shape)

In [ ]:
precios.info()

In [ ]:
precios.head()

## Merge de los DF

In [ ]:
df=pd.merge(ventas,calendario[["d","date","event2","event","yearweek"]],how="right",on="d") # Join ventas con calendario

In [ ]:
df["date"]=pd.to_datetime(df["date"])

In [ ]:
df.info()

In [ ]:
df.shape

In [ ]:
df.drop(["store"],axis=1,inplace=True) # eliminamos columna duplicada

In [ ]:
df.head()

In [ ]:
df.sort_values(["id", "date"], ascending = True, inplace = True)

In [ ]:
df["Cantidad"].sum()

In [ ]:
df=pd.merge(df,precios[["id","yearweek","sell_price"]],how="left",on=["id","yearweek"]) #join con precios

In [ ]:
df["Cantidad"].sum()

In [ ]:
df.info()

In [ ]:
df.isnull().sum()

In [ ]:
# rellenamos nulos con el valor anterior o posterior si no encuentra el anterior.
df["sell_price"] = df.groupby("id", group_keys = False)["sell_price"].apply(lambda series: series.bfill().ffill())

In [ ]:
df["sell_price"].isnull().sum() # verificamos nulos en sell price

In [ ]:
df.info()

In [ ]:
df.head()

In [ ]:
df.tail()

In [ ]:
from pathlib import Path  
filepath = Path("data/tablafinal.csv")  
filepath.parent.mkdir(parents=True, exist_ok=True)  
df.to_csv(filepath,index=False)  

In [4]:
df=pd.read_csv("data/tablafinal.csv")

In [5]:
df["date"]=pd.to_datetime(df["date"])

# Mapping DF

## Agrupamos por semana

In [6]:
columnasbase=df.columns[:6].tolist()
columnasbase.append("yearweek")
columnasbase

['id', 'item', 'category', 'department', 'store_code', 'region', 'yearweek']

In [7]:
ventassem=df.groupby(by=columnasbase).agg({"Cantidad":"sum","sell_price":"mean","event":"sum","event2":"sum","date":"max"}).reset_index()

In [8]:
ventassem.head()

,id,item,category,department,store_code,region,yearweek,Cantidad,sell_price,event,event2,date
0,ACCESORIES_1_001_BOS_1,0,0,0,0,0,201104,0,12.7414,0,0,2011-01-30
1,ACCESORIES_1_001_BOS_1,0,0,0,0,0,201105,0,12.7414,1,7,2011-02-06
2,ACCESORIES_1_001_BOS_1,0,0,0,0,0,201106,0,12.7414,0,0,2011-02-13
3,ACCESORIES_1_001_BOS_1,0,0,0,0,0,201107,0,12.7414,0,0,2011-02-20
4,ACCESORIES_1_001_BOS_1,0,0,0,0,0,201108,0,12.7414,0,0,2011-02-27


In [9]:
df.head()

,id,item,category,department,store_code,region,d,Cantidad,date,event2,event,yearweek,sell_price
0,ACCESORIES_1_001_BOS_1,0,0,0,0,0,d_1,0,2011-01-29,0,0,201104,12.7414
1,ACCESORIES_1_001_BOS_1,0,0,0,0,0,d_2,0,2011-01-30,0,0,201104,12.7414
2,ACCESORIES_1_001_BOS_1,0,0,0,0,0,d_3,0,2011-01-31,1,0,201105,12.7414
3,ACCESORIES_1_001_BOS_1,0,0,0,0,0,d_4,0,2011-02-01,1,0,201105,12.7414
4,ACCESORIES_1_001_BOS_1,0,0,0,0,0,d_5,0,2011-02-02,1,0,201105,12.7414


In [10]:
df['weekday'] = df['date'].dt.dayofweek

In [11]:
df['weekdaynam'] = df['date'].dt.day_name()

In [12]:
df.loc[(df['weekday'] >= 0) & (df['weekday'] <= 3), 'sales_mon_thu'] = df['Cantidad']

In [13]:
df.loc[(df['weekday'] == 4) | (df['weekday'] == 5) | (df['weekday'] == 6), 'sales_fri_sun'] = df['Cantidad']

In [14]:
df_grouped = df.groupby(['id', 'yearweek']).agg({'sales_mon_thu': 'sum', 'sales_fri_sun': 'sum'}).reset_index()

In [15]:
df_grouped.tail()

,id,yearweek,sales_mon_thu,sales_fri_sun
8476215,SUPERMARKET_3_827_PHI_3,201616,0.0,0.0
8476216,SUPERMARKET_3_827_PHI_3,201617,0.0,0.0
8476217,SUPERMARKET_3_827_PHI_3,201618,0.0,0.0
8476218,SUPERMARKET_3_827_PHI_3,201619,0.0,0.0
8476219,SUPERMARKET_3_827_PHI_3,201620,0.0,0.0


In [16]:
ventassem=pd.merge(ventassem,df_grouped[["id","yearweek","sales_mon_thu","sales_fri_sun"]],how="left",on=["id","yearweek"]) #join con precios

In [17]:
ventassem.shape

(8476220, 14)

In [18]:
ventassem.tail(20)

,id,item,category,department,store_code,region,yearweek,Cantidad,sell_price,event,event2,date,sales_mon_thu,sales_fri_sun
8476200,SUPERMARKET_3_827_PHI_3,3048,2,6,9,2,201601,10,1.2,0,0,2016-01-10,4.0,6.0
8476201,SUPERMARKET_3_827_PHI_3,3048,2,6,9,2,201602,17,1.2,0,0,2016-01-17,13.0,4.0
8476202,SUPERMARKET_3_827_PHI_3,3048,2,6,9,2,201603,10,1.2,0,0,2016-01-24,9.0,1.0
8476203,SUPERMARKET_3_827_PHI_3,3048,2,6,9,2,201604,12,1.2,0,0,2016-01-31,6.0,6.0
8476204,SUPERMARKET_3_827_PHI_3,3048,2,6,9,2,201605,9,1.2,1,7,2016-02-07,4.0,5.0
8476205,SUPERMARKET_3_827_PHI_3,3048,2,6,9,2,201606,17,1.2,0,0,2016-02-14,11.0,6.0
8476206,SUPERMARKET_3_827_PHI_3,3048,2,6,9,2,201607,9,1.2,0,0,2016-02-21,7.0,2.0
8476207,SUPERMARKET_3_827_PHI_3,3048,2,6,9,2,201608,9,1.2,0,0,2016-02-28,4.0,5.0
8476208,SUPERMARKET_3_827_PHI_3,3048,2,6,9,2,201609,15,1.2,0,0,2016-03-06,11.0,4.0
8476209,SUPERMARKET_3_827_PHI_3,3048,2,6,9,2,201610,8,1.2,0,0,2016-03-13,0.0,8.0


In [19]:
ventassem["Cantidad"].sum()

65695409

In [20]:
ventassem["sales_mon_thu"].sum()+ventassem["sales_fri_sun"].sum()

65695409.0

In [21]:
def calcular_percentage(cantidad, parte):
    if cantidad == 0:
        return 0
    else:
        return (parte / cantidad)

In [ ]:
#ventassem['sales_mon_fri_por']=ventassem["sales_mon_fri"] /ventassem["Cantidad"]

In [ ]:
#ventassem['sales_mon_fri_por'].fillna(0,inplace=True)

In [ ]:
#ventassem['sales_sat_sun_por']=ventassem["sales_sat_sun"]/ventassem["Cantidad"]

In [ ]:
#ventassem['sales_sat_sun_por'].fillna(0,inplace=True)

In [22]:
ventassem['sales_mon_thu_por'] = ventassem.apply(lambda row: calcular_percentage(row['Cantidad'], row['sales_mon_thu']), axis=1)

In [23]:
ventassem['sales_fri_sun_por'] = ventassem.apply(lambda row: calcular_percentage(row['Cantidad'], row['sales_fri_sun']), axis=1)

In [25]:
ventassem["week"]=ventassem['date'].dt.isocalendar().week
groupedweek=ventassem.groupby(["id","week"]).agg({"sales_mon_thu_por":"mean","sales_fri_sun_por":"mean"}).reset_index()
groupedweek = groupedweek.rename(columns={'sales_mon_thu_por': 'sales_mon_thu_por_mean', 'sales_fri_sun_por': 'sales_fri_sun_por_mean'})
ventassem=pd.merge(ventassem,groupedweek, on=["id","week"],how="left")


In [26]:
ventassem.head()

,id,item,category,department,store_code,region,yearweek,Cantidad,sell_price,event,event2,date,sales_mon_thu,sales_fri_sun,sales_mon_thu_por,sales_fri_sun_por,week,sales_mon_thu_por_mean,sales_fri_sun_por_mean
0,ACCESORIES_1_001_BOS_1,0,0,0,0,0,201104,0,12.7414,0,0,2011-01-30,0.0,0.0,0.0,0.0,4,0.166667,0.166667
1,ACCESORIES_1_001_BOS_1,0,0,0,0,0,201105,0,12.7414,1,7,2011-02-06,0.0,0.0,0.0,0.0,5,0.250000,0.083333
2,ACCESORIES_1_001_BOS_1,0,0,0,0,0,201106,0,12.7414,0,0,2011-02-13,0.0,0.0,0.0,0.0,6,0.166667,0.166667
3,ACCESORIES_1_001_BOS_1,0,0,0,0,0,201107,0,12.7414,0,0,2011-02-20,0.0,0.0,0.0,0.0,7,0.166667,0.333333
4,ACCESORIES_1_001_BOS_1,0,0,0,0,0,201108,0,12.7414,0,0,2011-02-27,0.0,0.0,0.0,0.0,8,0.125000,0.375000


In [27]:
from pathlib import Path  
filepath = Path("data/ventassem.csv")  
filepath.parent.mkdir(parents=True, exist_ok=True)  
ventassem.to_csv(filepath,index=False)  

# EDA

## Graficos

In [ ]:
df.groupby(["region"])["Cantidad"].sum().sort_values(ascending = False).plot(kind = "bar", figsize = (12, 4))

In [ ]:
(
    df.
    groupby(["store_code"])
    ["Cantidad"].sum()
    .sort_values(ascending = False)
    .plot(kind = "bar", figsize = (12, 4))
);

In [ ]:
(
    df.
    groupby(["category"])
    ["Cantidad"].sum()
    .sort_values(ascending = False)
    .plot(kind = "bar", figsize = (12, 4))
);

In [ ]:
(
    df.
    groupby(["department"])
    ["Cantidad"].sum()
    .sort_values(ascending = False)
    .plot(kind = "bar", figsize = (12, 4))
);

In [ ]:
y = df.set_index("date").resample("M")["Cantidad"].sum()

In [ ]:
fig = plt.figure(figsize = (12, 10))
ax = fig.subplots(1, 1)
ax.plot(y)
    
plt.suptitle(t = "Ventas Totales", fontsize = 20)